In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
import glob
import matplotlib.pyplot as plt

In [2]:
path = "C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Host\\"
all_files = glob.glob(path + "*.csv")
all_files

['C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Host\\db_oracle_11g.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Host\\dcos_container.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Host\\dcos_docker.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Host\\mw_redis.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Host\\os_linux.csv']

In [3]:
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    dfs.append(df)
df = pd.concat(dfs, axis=0, ignore_index=True)


In [4]:
anomaly_time = 1590529140000
# 1590430140000 db 007
# 1590431940000 docker 008
# 1590433740000 os 21
# 1590435540000 os 18
df = df[(df.timestamp >= (anomaly_time-1200000)) & (df.timestamp <= (anomaly_time + 120000))]
df

,itemid,name,bomc_id,timestamp,value,cmdb_id
146322,999999998651280,CPU_free_pct,ZJ-002-056,1590527940000,98.133749,db_008
146323,999999998650680,CPU_free_pct,ZJ-002-056,1590527940000,99.401682,db_001
146324,999999998651100,MEM_real_util,ZJ-002-053,1590527940000,80.450000,db_007
146325,999999998650980,CPU_free_pct,ZJ-002-056,1590527940000,98.802366,db_003
146326,999999996381601,CPU_free_pct,ZJ-002-056,1590527941000,97.540565,db_009
...,...,...,...,...,...,...
558272,999999996432240,Disk_rd_ios,ZJ-001-098,1590529260000,0.000000,os_007
558273,65435640,Disk_io_util,ZJ-001-125,1590529260000,0.000000,os_020
558274,65439900,Disk_io_util,ZJ-001-125,1590529260000,2.000000,os_012
558275,63308940,Outgoing_network_traffic,ZJ-001-072,1590529260000,0.021174,os_022


In [2]:
save_path ='D:\\THU Studies\\Advance Network Management\\Project\\Anomaly-detection\\local_data\\noJDBC\\'
df.to_csv(save_path+'kpi_data_527_os21_sentque.csv', index = False)

NameError: name 'df' is not defined

In [2]:
path = "C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Trace\\"
all_files = glob.glob(path + "*.csv")
all_files

['C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Trace\\trace_csf.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Trace\\trace_fly_remote.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Trace\\trace_local.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Trace\\trace_osb.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_27\\Trace\\trace_remote_process.csv']

In [3]:
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    dfs.append(df)
# df = pd.concat(dfs, axis=0, ignore_index=True)

In [24]:
df.cmdb_id.unique()

array(['docker_002', 'docker_005', 'docker_001', 'docker_006',
       'docker_003', 'docker_007', 'docker_008', 'docker_004'],
      dtype=object)

In [4]:
for df in dfs:
    if df['callType'].iloc[0]=='JDBC' or df['callType'].iloc[0]=='LOCAL':
        df['serviceName'] = df['dsName']
    elif df['callType'].iloc[0]=='RemoteProcess' or df['callType'].iloc[0]=='OSB':
        df['serviceName'] = df['cmdb_id']

In [5]:
# df.head()

In [5]:
df = pd.concat(dfs, axis=0, ignore_index=True)
# df.head()

In [6]:
df = df.drop('dsName', axis=1, index=None)
# df = df.drop(['path'], axis=1)
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
0,CSF,1590508800030,2308.0,True,73271d2485849471c76f,7327121d411594e1c76f,7327120256949471c76f,os_022,csf_001
1,CSF,1590508800033,251.0,True,73271854e26857b1c76f,73271e1ee0985712c76f,73271e287b7857b1c76f,os_022,csf_001
2,CSF,1590508800034,884.0,True,73271c5c4a2367b1c76f,732717e236736722c76f,732714be573367b1c76f,os_022,csf_001
3,CSF,1590508800035,554.0,True,732712b99f4257b1c76f,73271bc681725732c76f,732714a7505257b1c76f,os_022,csf_001
4,CSF,1590508800036,882.0,True,7327133bf44367f1c76f,732711fddf936742c76f,7327100cbd5367f1c76f,os_022,csf_001


In [9]:
df.cmdb_id.unique()

array(['os_022', 'docker_002', 'docker_001', 'os_021', 'docker_003',
       'docker_004', nan, 'docker_008', 'docker_007', 'docker_005',
       'docker_006'], dtype=object)

In [7]:
df.dropna(inplace = True, axis=0)

In [9]:
anomaly_time = 1590529140000
# 1590430140000 db 007
# 1590431940000 docker 008
# 1590433740000 os 21
df = df[(df.startTime >= (anomaly_time-1200000)) & (df.startTime <= (anomaly_time + 120000))]
# df.to_csv(save_path + 'trace_data_527_os21_sentque.csv')

In [10]:
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
710638,CSF,1590527940030,445.0,True,9327134ba800267b98a1,93271f208f3026eb98a1,932710c86210267b98a1,os_022,csf_001
710639,CSF,1590527940030,489.0,True,93271c735781637b98a1,93271b84437263eb98a1,932717e9e191637b98a1,os_022,csf_001
710640,CSF,1590527940031,515.0,True,93271790d84263ab98a1,93271ceacc9263fb98a1,932716cc5d5263ab98a1,os_022,csf_001
710641,CSF,1590527940032,3274.0,True,93271cabdf43166b98a1,93271c077693160c98a1,932715ad1a53166b98a1,os_022,csf_001
710642,CSF,1590527940033,3273.0,True,93271dde5e63167b98a1,93271600c614161c98a1,93271ffd0c73167b98a1,os_022,csf_001


In [50]:
ldf = df[df.callType=='LOCAL']

In [65]:
ldf[ldf.serviceName=='db_007'].sort_values('startTime').tail(60)

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
2315590,LOCAL,1590517473980,5.0,False,8327148d408633b16da7,83271257910722cb6da7,832712f90d0912226da7,docker_002,db_007
2315595,LOCAL,1590517474028,89.0,False,8327199c5b6533a32da7,83271ab60f7719ce6da7,832710f8207098692da7,docker_001,db_007
2315651,LOCAL,1590517474202,91.0,False,832711f66f4788551da7,83271656703197a97da7,83271a1bb69257b51da7,docker_003,db_007
2315653,LOCAL,1590517474204,91.0,False,832716f8604788c81da7,83271a05dd0297c97da7,83271cddf793576b1da7,docker_003,db_007
2315657,LOCAL,1590517474221,95.0,False,83271e4ce54923404da7,832712ae873329da7da7,83271bde727698524da7,docker_001,db_007
2315676,LOCAL,1590517474273,85.0,False,832719afb360331e6da7,83271b430754421e7da7,83271040c167227e6da7,docker_002,db_007
2315703,LOCAL,1590517474376,80.0,False,832717c6728143cd6da7,83271ce2824252848da7,83271d7eae27224e6da7,docker_002,db_007
2315704,LOCAL,1590517474377,81.0,False,8327192b3221339f6da7,83271db0546252948da7,83271c65f78722407da7,docker_002,db_007
2315719,LOCAL,1590517474425,4.0,False,83271c818c0033405da7,832711ca467039978da7,83271259427309155da7,docker_001,db_007
2315743,LOCAL,1590517474497,17.0,False,8327190da70888528da7,83271ff0f472151c8da7,83271eb8c15405f28da7,docker_004,db_007


In [11]:
df1 = df[df['callType']=='RemoteProcess']
df1 = df1[['pid','cmdb_id']]
df1 = df1.set_index('pid')

In [12]:
elapse_time = {}
children = {}

def do_thing(row):
    if row['id'] in df1.index:
        row['serviceName'] = df1.at[row['id'],'cmdb_id']
    if row['pid'] != 'None':
        children[row['pid']] = children.get(row['pid'], [])
        children[row['pid']].append(row['id'])
    elapse_time[row['id']] = float(row['elapsedTime'])
    return row
df = df.apply(do_thing, axis=1)


df['actual_time'] = 0.0

def get_actual_time(row):
    total_child = 0.0
    if row['id'] in children:
        for child in children[row['id']]:
            total_child += elapse_time[child]
    row['actual_time'] = row['elapsedTime'] - total_child
    return row

df = df.apply(get_actual_time, axis = 1)

df = df[~(df['serviceName'].str.contains('csf', na=True))]

In [13]:
df = df.sort_values(['traceId', 'startTime'], ignore_index=True)

In [14]:
df.head(60)

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName,actual_time
0,OSB,1590528486598,243.0,True,932710000a04776c0e22,93271f2d4014776c0e22,None,os_022,os_022,17.0
1,CSF,1590528486610,226.0,True,932710000a04776c0e22,93271fab5234772d0e22,93271f2d4014776c0e22,os_022,docker_001,2.0
2,RemoteProcess,1590528486610,224.0,True,932710000a04776c0e22,93271fa1a805212d0e22,93271fab5234772d0e22,docker_001,docker_001,17.0
3,FlyRemote,1590528486613,7.0,True,932710000a04776c0e22,93271b96aa25215d0e22,93271fa1a805212d0e22,docker_001,fly_remote_001,7.0
4,CSF,1590528486621,35.0,True,932710000a04776c0e22,93271ecf5c6521dd0e22,93271fa1a805212d0e22,docker_001,docker_005,3.0
5,RemoteProcess,1590528486623,32.0,True,932710000a04776c0e22,932715d6312611fd0e22,93271ecf5c6521dd0e22,docker_005,docker_005,25.0
6,LOCAL,1590528486643,7.0,True,932710000a04776c0e22,932712567e48113f0e22,932715d6312611fd0e22,docker_005,db_003,7.0
7,CSF,1590528486657,23.0,True,932710000a04776c0e22,93271254176621101e22,93271fa1a805212d0e22,docker_001,docker_006,3.0
8,RemoteProcess,1590528486658,20.0,True,932710000a04776c0e22,932714fa537464201e22,93271254176621101e22,docker_006,docker_006,11.0
9,LOCAL,1590528486662,6.0,True,932710000a04776c0e22,932711ed891564601e22,932714fa537464201e22,docker_006,db_003,6.0


In [16]:
save_path ='D:\\THU Studies\\Advance Network Management\\Project\\Anomaly-detection\\local_data\\esd\\'
df.to_csv(save_path+'trace_528347_os18.csv', index = False)

In [ ]:
from tqdm import tqdm
for index, row in tqdm(df.iterrows()):
    if row['id'] in csf_cmdb:
        df.at[index, 'serviceName'] = csf_cmdb[row['id']]

In [ ]:
elapse_time = {}
children = {}
for index, row in tqdm(df.iterrows()):
    if row['pid'] != 'None':
        if row['pid'] in children.keys():
            children[row['pid']].append(row['id'])
        else:
            children[row['pid']] = [row['id']]
    elapse_time[row['id']] = float(row['elapsedTime'])

In [ ]:
df['actual_time'] = 0.0
for index, row in tqdm(df.iterrows()):
    total_child = 0.0
    if row['id'] not in children.keys():
        df.at[index, 'actual_time'] = row['elapsedTime']
        continue
    for child in children[row['id']]:
        total_child += elapse_time[child]
    df.at[index, 'actual_time'] = row['elapsedTime'] - total_child

In [ ]:
df = df.sort_values(['traceId','startTime','callType'], ignore_index = True)

In [ ]:
df.serviceName.unique()

In [ ]:
df.head(60)

In [14]:
grouped_df = df.groupby(['cmdb_id', 'serviceName'])[['startTime','actual_time']]

In [15]:
grouped_df.groups.keys()

dict_keys([('docker_001', 'db_007'), ('docker_001', 'db_009'), ('docker_001', 'docker_001'), ('docker_001', 'docker_005'), ('docker_001', 'docker_006'), ('docker_001', 'fly_remote_001'), ('docker_002', 'db_007'), ('docker_002', 'db_009'), ('docker_002', 'docker_002'), ('docker_002', 'docker_005'), ('docker_002', 'docker_006'), ('docker_002', 'fly_remote_001'), ('docker_003', 'db_007'), ('docker_003', 'db_009'), ('docker_003', 'docker_003'), ('docker_003', 'docker_007'), ('docker_003', 'docker_008'), ('docker_003', 'fly_remote_001'), ('docker_004', 'db_007'), ('docker_004', 'db_009'), ('docker_004', 'docker_004'), ('docker_004', 'docker_007'), ('docker_004', 'docker_008'), ('docker_004', 'fly_remote_001'), ('docker_005', 'db_003'), ('docker_005', 'docker_005'), ('docker_006', 'db_003'), ('docker_006', 'docker_006'), ('docker_007', 'db_003'), ('docker_007', 'docker_007'), ('docker_007', 'fly_remote_001'), ('docker_008', 'db_003'), ('docker_008', 'docker_008'), ('docker_008', 'fly_remote_

In [16]:
def esd_test_statistics(x, hybrid=True):
  """
  Compute the location and dispersion sample statistics used to carry out the ESD test.
  """
  if hybrid:
    location = pd.Series(x).median(skipna=True) # Median
    dispersion = np.median(np.abs(x - np.median(x))) # Median Absolute Deviation
  else:  
    location = pd.Series(x).mean(skipna=True) # Mean
    dispersion = pd.Series(x).std(skipna=True) # Standard Deviation
    
  return location, dispersion    

In [17]:
# def seasonal_mean(x, freq):
#   """
#   Returns the mean of the timeseries for each period in x.
  
#   x    : List, array, or series containing the time series
#   freq : Int that gives the number of periods per cycle (7 for week, 12 for monthly, etc)
#   """
#   return np.array([pd.Series(x[i::freq]).mean(skipna=True) for i in range(freq)])

def ts_S_Md_decomposition(x):
  """
  Decomposes the timeseries using a modified STL method:Rx = X - Sx - X^~.
    Rx : Residuals
    X  : Original time series
    Sx : Seasonality component (average value per period)
    X^~: Median of original timeseries
    
  x    : List, array, or series containing the time series 
  freq : Int that gives the number of periods per cycle (7 for week, 12 for monthly, etc)  
  """
  nobs = len(x)
  
  # Seasonality
#   period_averages = seasonal_mean(x, freq)
#   seasonal = np.tile(period_averages, nobs // freq + 1)[:nobs]
  
  # Median
  med = np.tile(pd.Series(x).median(skipna=True), nobs)
  
  # Residuals
#   res = np.array(x) - seasonal - med
  res = np.array(x) - med
  return {"observed": np.array(x), "median":med, "residual":res}

In [68]:
def esd_test(x, alpha=0.95, ub=0.499, hybrid=True):
  """
  Carries out the Extreme Studentized Deviate(ESD) test which can be used to detect one or more outliers present in the timeseries
  
  x      : List, array, or series containing the time series
  freq   : Int that gives the number of periods per cycle (7 for week, 12 for monthly, etc)
  alpha  : Confidence level in detecting outliers
  ub     : Upper bound on the fraction of datapoints which can be labeled as outliers (<=0.499)
  hybrid : Whether to use the robust statistics (median, median absolute error) or the non-robust versions (mean, standard deviation) to test for anomalies
  """
  nobs = len(x)
  if ub > 0.4999:
    ub = 0.499
  k = max(int(np.floor(ub * nobs)), 1) # Maximum number of anomalies. At least 1 anomaly must be tested.
#   res_tmp = ts_S_Md_decomposition(x)["residual"] # Residuals from time series decomposition
    
  # Carry out the esd test k times  

  res = np.ma.array(x, mask=False) # The "ma" structure allows masking of values to exclude the elements from any calculation
  anomalies = [] # returns the indices of the found anomalies
  med = np.median(x)
  ctr = 0
  for i in range(1, k+1):
      location, dispersion = esd_test_statistics(res, hybrid) # Sample statistics
      tmp = np.abs(res - location) / dispersion
      idx = np.argmax(tmp) # Index of the test statistic
      test_statistic = tmp[idx]
      n = nobs - res.mask.sum() # sums  nonmasked values
      critical_value = (n - i) * t.ppf(alpha, n - i - 1) / np.sqrt((n - i - 1 + np.power(t.ppf(alpha, n - i - 1), 2)) * (n - i - 1)) 
      if test_statistic > critical_value:
          ctr += 1
          anomalies.append((x[idx]-med) / med)
          # anomalies.append(test_statistic)
      res.mask[idx] = True
  if len(anomalies) == 0:
      return 0
  # return np.mean(np.abs(anomalies))
  return ctr

In [22]:
grouped_df.groups.keys()

dict_keys([('docker_001', 'db_007'), ('docker_001', 'db_009'), ('docker_001', 'docker_001'), ('docker_001', 'docker_005'), ('docker_001', 'docker_006'), ('docker_001', 'fly_remote_001'), ('docker_002', 'db_007'), ('docker_002', 'db_009'), ('docker_002', 'docker_002'), ('docker_002', 'docker_005'), ('docker_002', 'docker_006'), ('docker_002', 'fly_remote_001'), ('docker_003', 'db_007'), ('docker_003', 'db_009'), ('docker_003', 'docker_003'), ('docker_003', 'docker_007'), ('docker_003', 'docker_008'), ('docker_003', 'fly_remote_001'), ('docker_004', 'db_007'), ('docker_004', 'db_009'), ('docker_004', 'docker_004'), ('docker_004', 'docker_007'), ('docker_004', 'docker_008'), ('docker_004', 'fly_remote_001'), ('docker_005', 'db_003'), ('docker_005', 'docker_005'), ('docker_006', 'db_003'), ('docker_006', 'docker_006'), ('docker_007', 'db_003'), ('docker_007', 'docker_007'), ('docker_007', 'fly_remote_001'), ('docker_008', 'db_003'), ('docker_008', 'docker_008'), ('docker_008', 'fly_remote_

In [22]:
class RCA():
    def local_initiate(self):
        self.dockers = ['docker_001', 'docker_002', 'docker_003', 'docker_004',
                'docker_005', 'docker_006', 'docker_007', 'docker_008']
        self.docker_hosts = ['os_017', 'os_018', 'os_019', 'os_020']

        self.docker_kpi_names = ['container_cpu_used', None]
        self.os_kpi_names = ['Sent_queue', 'Received_queue']
        self.db_kpi_names = ['Proc_User_Used_Pct','Proc_Used_Pct','Sess_Connect','On_Off_State', 'tnsping_result_time']

        self.docker_lookup_table = {}
        for i in range(len(self.dockers)):
            self.docker_lookup_table[self.dockers[i]] = self.docker_hosts[i % 4]
            
    def find_anomalous_rows(self, min_threshold = 5):
        table = self.anomaly_chart.copy()
        threshold = max( 0.5 * table.stack().max(), min_threshold)
        dodgy_rows = []
        just_rows = []
        for column in table:
            v = 0
            r = ''
            for index, row in table.iterrows():
                if (row[column] > threshold):
                    if index == column:
                        dodgy_rows.append([index, row[column]])
                        just_rows.append(index)
                        break
                    elif (row[column] > v):
                        v = row[column]
                        r = index
            if r != '':
                dodgy_rows.append([r, column, v])
                just_rows.append(r)
        
        output = self.localize(dodgy_rows, list(set(just_rows)))
        return output


    def find_anomalous_kpi(self, cmdb_id):
        kpi_names = []
        if 'os' in cmdb_id:
            kpi_names = self.os_kpi_names
        elif 'docker' in cmdb_id:
            kpi_names = self.docker_kpi_names
        else:
            kpi_names = self.db_kpi_names

        return kpi_names


    def localize(self, dodgy_rows, just_rows):
        n = len(just_rows)
        if n < 1:
            return None
        if n == 1:
            KPIs = self.find_anomalous_kpi(just_rows[0])
            to_be_sent = []
            for KPI in KPIs:
                to_be_sent.append([just_rows[0], KPI])
            return to_be_sent
        if n == 2:
            r0 = just_rows[0]
            r1 = just_rows[1]
            if ('os' in r0) and ('os' in r1):
                KPI = self.find_anomalous_kpi('os_001')
                return [['os_001', KPI[0]],['os_001', KPI[1]]]
            elif ('docker' in r0) and ('docker' in r1):
                if self.docker_lookup_table[r0] == self.docker_lookup_table[r1]:
                    KPI = self.find_anomalous_kpi(self.docker_lookup_table[r0])
                    return [[self.docker_lookup_table[r0], KPI]]
            else:
                KPI0s = self.find_anomalous_kpi(r0)
                KPI1s = self.find_anomalous_kpi(r1)
                to_be_sent = []
                for kpi in KPI0s:
                    to_be_sent.append([r0, kpi])
                for kpi in KPI1s:
                    to_be_sent.append([r1, kpi])
                return to_be_sent
        if n > 2:
            dodgy_rows.sort(key = lambda x: x[2], reverse = True)
            just_rows = [x[0] for x in dodgy_rows]
            just_rows = list(set(just_rows))
            return self.localize(dodgy_rows[:2], just_rows[:2])

In [19]:
rca = RCA()
rca.local_initiate()
rca.anomaly_chart = anomaly_chart
output = rca.find_anomalous_rows()

NameError: name 'anomaly_chart' is not defined

In [ ]:
output

In [73]:
anomaly_chart = pd.DataFrame()
for (a, b), value in grouped_df:
    value['time_group'] = value.startTime//60000
    # value['date'] = pd.to_datetime(value.startTime, unit='ms') + pd.Timedelta('08:00:00')
    # value = value.set_index(pd.DatetimeIndex(value['date']))
    # value['rollingTime'] = df.rolling(20,'1min').actual_time.mean()
    value = value.groupby(['time_group'])['actual_time'].mean().reset_index()
    result = esd_test(value['actual_time'].to_numpy(), alpha=0.95, ub=0.4, hybrid=True)
    anomaly_chart.loc[b,a] = result
    # print(a,b,' = ',len(result)/len(value)*100)

In [74]:
anomaly_chart.sort_index(inplace=True)
anomaly_chart

,docker_001,docker_002,docker_003,docker_004,docker_005,docker_006,docker_007,docker_008,os_021,os_022
db_003,NaN,NaN,NaN,NaN,25.0,45.0,40.0,41.0,NaN,NaN
db_007,36.0,43.0,39.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN
db_009,42.0,42.0,41.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN
docker_001,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0
docker_002,NaN,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0
docker_003,NaN,NaN,43.0,NaN,NaN,NaN,NaN,NaN,39.0,NaN
docker_004,NaN,NaN,NaN,53.0,NaN,NaN,NaN,NaN,42.0,NaN
docker_005,42.0,39.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN
docker_006,42.0,38.0,NaN,NaN,NaN,32.0,NaN,NaN,NaN,NaN
docker_007,NaN,NaN,34.0,43.0,NaN,NaN,35.0,NaN,NaN,NaN


In [ ]:
test = grouped_df.get_group(('docker_008', 'docker_008'))

In [ ]:
x = test[['startTime','actual_time']]
# x_val = x['actual_time'].to_numpy()
# x

In [ ]:
x['time_group'] = x.startTime//10000
x = x.groupby(['time_group'])['actual_time'].mean().reset_index()

In [ ]:
x_val = x['actual_time'].to_numpy()

In [ ]:
result = esd_test(x_val, freq=3, alpha=0.99, ub=0.4, hybrid=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pd.Series(x_val).index, x_val, color="blue", label = "Original")
ax.scatter(result, x_val[result], color='red', label='Anomaly')
plt.legend(loc="best")
plt.show()

In [ ]:
x = test[['startTime','actual_time']]
# x_val = x['actual_time'].to_numpy()
# x
x['time_group'] = x.startTime//1000
x = x.groupby(['time_group'])['actual_time'].mean().reset_index()

In [ ]:
x_test = x.copy()
x_test.index = pd.to_datetime(x_test['time_group'], unit="s")

In [ ]:
x_test